In [1]:
!pip install --quiet geocoder
print('pip install geocoder complete.')

pip install geocoder complete.


In [2]:
import geocoder
import os.path
import pandas as pd

CODED = '/kaggle/working/coded.csv'
FIXES = {
    'UniversitÃ© PSL': 'Universite PSL',
    'Fudan University': 'National Fudan University'
}
WORLD = '/kaggle/input/top-100-universities-in-the-world-qs-ranking/top 100 world university 2024.csv'

def get(arg: str):
    json_result = geocoder.arcgis(location=arg).json
    if json_result:
        result = dict(json_result)
        return result['lat'], result['lng']
    else:
        return 0, 0

if os.path.isfile(path=CODED):
    print('reading geocoded data from {}'.format(CODED))
    df = pd.read_csv(filepath_or_buffer=CODED, index_col=[0])
else:
    print('reading raw data from {}'.format(WORLD))
    df = pd.read_csv(filepath_or_buffer=WORLD, index_col=[0])
    df['fixed_university'] = [item if item not in FIXES.keys() else FIXES[item] for item in df['university'].values]
    print('geocoding universities')
    df['coordinates'] = df['fixed_university'].apply(func=get)
    df['latitude'], df['longitude'] = zip(*df['coordinates'])
    df = df.drop(columns=['fixed_university', 'coordinates'])
    df.to_csv(path_or_buf=CODED)
df.head()

reading raw data from /kaggle/input/top-100-universities-in-the-world-qs-ranking/top 100 world university 2024.csv
geocoding universities


,rank,university,overall_score,academic_reputation,employer_reputation,faculty_student_ratio,citations_per_faculty,international_faculty_ratio,international_students_ratio,international_research_network,employment_outcomes,sustainability,latitude,longitude
0,1,Massachusetts Institute of Technology (MIT),100.0,100.0,100.0,100.0,100.0,100.0,88.2,94.3,100.0,95.2,42.35897,-71.09350
1,2,University of Cambridge,99.2,100.0,100.0,100.0,92.3,100.0,95.8,99.9,100.0,97.3,52.20525,0.11779
2,3,University of Oxford,98.9,100.0,100.0,100.0,90.6,98.2,98.2,100.0,100.0,97.8,51.75467,-1.25489
3,4,Harvard University,98.3,100.0,100.0,98.3,100.0,84.6,66.8,100.0,100.0,96.7,42.37588,-71.12340
4,5,Stanford University,98.1,100.0,100.0,100.0,99.9,99.9,51.2,95.8,100.0,94.4,37.42907,-122.16978


In [3]:
from plotly.express import histogram
for column in df.columns[2:-2]:
    histogram(data_frame=df, x=column).show()

In [4]:
from plotly.express import scatter_geo
scatter_geo(data_frame=df, lat='latitude', lon='longitude', hover_name='university', color='overall_score')